In [32]:
import pandas as pd
import matplotlib.pyplot as plt
from pylab import *
from scipy.optimize import curve_fit
from scipy import stats
import scipy.cluster.hierarchy as sch
from scipy.stats import norm
from nested_lookup import nested_lookup
import json 
from citableclass import Citable
import warnings

In [33]:
def front(self, n):
    return self.iloc[:, :n]
pd.DataFrame.front = front
def back(self, n):
    return self.iloc[:, -n:]
pd.DataFrame.back = back

## Current dataset 

In [21]:
%%capture
%run Part4_Modeling.ipynb
#declination = Run(iterationsteps=200, threshold=157, bins=12, mods=3, sigma=0.2, startvalue=0.5);
#with open('current_data.json', 'w') as outfile:
#    json.dump(declination, outfile)

In [34]:
%%capture
declination = Run(iterationsteps=550, threshold=55, bins=12, mods=3, sigma=0.2, startvalue=0.5);
#with open('current_data.json', 'w') as outfile:
#    json.dump(declination, outfile)

In [5]:
#with open('current_data.json') as f:
#    declinationjson = json.load(f)

In [35]:
def FirstFit(data=""):
    data=data
    print(data)
#    #data = [item for sublist in l for item in sublist]
    if len(data)>0:
        bins=20
        # best fit of data
        (mu, sigma) = norm.fit(data)
        # the histogram of the data
        n, bins, patches = plt.hist(data,  bins=bins, normed=1, facecolor='green', alpha=0.75)
        # add a 'best fit' line
        y = mlab.normpdf(bins, mu, sigma)
        #l = plt.plot(bins, y, 'r--', linewidth=2)
        #plot
        return bins, y, mu, sigma

In [36]:
%%capture
distributions=pd.DataFrame.from_dict(declination)
flatlist=list()
for objects in list(distributions.keys()):
    
    for element in ["1.0", "2.0", "3.0", "4.0", "5.0", "6.0", "7.0", "8.0", "9.0", "10.0"]:
        try:
            if len(distributions[objects]["azimuth"][element])>0:
                
                azimuth=[i[0] for i in distributions[objects]["azimuth"][element]]
                declinations= [i[0] for i in distributions[objects]["declination"][element]]
                azimuth_mean=np.mean(azimuth)
                declination_mean=np.mean(declinations)
                latitude=distributions[objects]["latitude"]
                longitude=distributions[objects]["longitude"]
                binsdec, ydec, mudec, sigmadec = FirstFit(declinations)
                binsazi, yazi, muazi, sigmaazi = FirstFit(azimuth)
                number_elements = distributions[objects]["declination"][element][0][1]
                azimuth_michel = distributions[objects]["declination"][element][0][2]
                flatlist.append((element, objects, latitude, longitude, azimuth, yazi, binsazi, muazi, sigmaazi, declinations, ydec, binsdec, mudec, sigmadec, number_elements, azimuth_michel))
        except:
            pass
df=pd.DataFrame(flatlist)
df=df.rename(columns={0:'gate', 1: 'object', 2: "latitude", 3:'longitude', 4:'azimuth', 5: "prob (az)",  6: "bins (az)", 7: "mean azimuth", 8: "sigma azimuth", 9:'declination', 10: "prob (dec)", 11: "bins (dec)", 12: "mean declination",13: "sigma declination", 14: "number elements", 15:"azimuth(michel)"})
#df.head()

In [24]:
#distributions["QuedlinburgI"]

In [37]:
df.head()

,gate,object,latitude,longitude,azimuth,prob (az),bins (az),mean azimuth,sigma azimuth,declination,prob (dec),bins (dec),mean declination,sigma declination,number elements,azimuth(michel)
0,1.0,QuedlinburgI,51.813308,11.178354,"[330.0737, 315.47189, 324.98327, 331.34081, 32...","[0.0006139203391661239, 0.0011889488250392904,...","[288.72932, 291.27250499999997, 293.8156899999...",319.090246,10.579579,"[32.39794, 26.15093, 30.41884, 32.85302, 29.95...","[0.00046920449252568124, 0.0009831401130030691...","[11.44989, 12.6479035, 13.845917, 15.043930499...",27.466094,4.996915,5,"[[134, 136], [21, 23], [214, 217], [312, 313]]"
1,2.0,QuedlinburgI,51.813308,11.178354,"[338.80694, 351.6713, 357.19346, 358.21324, 33...","[5.147966062144608e-07, 2.1168978045825936e-06...","[309.46452, 311.990956, 314.517392, 317.043828...",349.067301,8.276810,"[35.1987, 37.71293, 38.13266, 38.16478, 32.801...","[4.323376171997256e-16, 1.5811124180992993e-14...","[23.13784, 23.8902825, 24.642725, 25.3951675, ...",36.942658,1.675503,1,"[[134, 136], [21, 23], [214, 217], [312, 313]]"
2,3.0,QuedlinburgI,51.813308,11.178354,"[32.10405, 38.99056, 39.54892, 18.9073, 39.515...","[0.0029551077408366227, 0.005017051546686616, ...","[6.71438, 9.859362, 13.004344, 16.149326000000...",33.557735,12.253642,"[31.58006, 28.7191, 28.47025, 35.79373, 28.485...","[0.00011280739583580588, 0.0002773957351650861...","[12.43632, 13.7084165, 14.980513, 16.2526095, ...",30.361982,4.944115,2,"[[134, 136], [21, 23], [214, 217], [312, 313]]"
3,5.0,QuedlinburgI,51.813308,11.178354,"[191.83126, 249.08304, 233.07714, 192.28796, 2...","[0.004605081805550003, 0.007324304636198494, 0...","[180.15635, 184.4572445, 188.758139, 193.05903...",209.343037,15.829816,"[-37.23547, -12.75107, -21.80152, -37.16141, -...","[0.035072286665202464, 0.04757174355168819, 0....","[-38.18652, -36.395399000000005, -34.604278, -...",-31.435745,5.865522,1,"[[134, 136], [21, 23], [214, 217], [312, 313]]"
4,6.0,QuedlinburgI,51.813308,11.178354,"[278.90264, 326.36338, 280.41582, 299.95191, 2...","[0.014639293121970586, 0.01962312354508429, 0....","[270.03975, 272.8559315, 275.672113, 278.48829...",285.127550,11.466619,"[5.49012, 30.97839, 6.41733, 17.9787, 11.6949,...","[0.023621300688798093, 0.031507530407419826, 0...","[0.02457, 1.572261, 3.119952, 4.66764299999999...",9.129168,6.690051,1,"[[134, 136], [21, 23], [214, 217], [312, 313]]"


In [38]:
citen = Citable("values_n550_th55_sig0.2_start0.5.json", formats="local", project="kreisgrabenanlage")
citen.write(df)

Please add metadata:
Wrote metadata file /home/gordon.fischer/ResearchCloud/Projects/kreisgrabenanlage/data/values_n550_th55_metadata.json
Please add documentation:
Wrote documentation file /home/gordon.fischer/ResearchCloud/Projects/kreisgrabenanlage/data/values_n550_th55_documentation.json


## Create flat dataframe

In [ ]:
citenew = Citable("Azimuth.json", formats="local")
citenew1= Citable("AllParameter.json", formats="local")

In [ ]:
citenew.write(df.front(8))

In [ ]:
citenew1.write(df)

In [ ]:
citenew.documentation()

In [ ]:
df=citenew.digitalresource()

## Fit procedure

### Plot prob-values for azimuth

In [ ]:
plt.figure(figsize=(17,10))
overall=list()
overall_azi=list()
overall_bins=list()
for element in range(len(df['prob (az)'])):
    overall.append((df["bins (az)"][element],df['prob (az)'][element] ))
    overall_azi.append(list(df['prob (az)'][element]))
    overall_bins.append(list(df["bins (az)"][element]))
    plt.title("uncorrected probability for azimuth", fontsize=15)
    plt.xlabel("azimuth [°]", fontsize=15)
    plt.ylabel("prob", fontsize=15)
    plt.plot(df["bins (az)"][element], df['prob (az)'][element])
    plt.hist(df["bins (az)"][element], color="green", weights=df['prob (az)'][element], bins=5, stacked=True)

### Plot prob-values for declination

In [ ]:
#df = citenew1.digitalresource()
#plt.figure(figsize=(17,10))
#overall=list()
#overall_dec=list()
#overall_bins=list()
#for element in range(len(df['prob (dec)'])):
#    overall.append((df["bins (dec)"][element],df['prob (dec)'][element] ))
#    overall_dec.append(list(df['prob (dec)'][element]))
#    overall_bins.append(list(df["bins (dec)"][element]))
#    plt.title("uncorrected probability for declination", fontsize=15)
#    plt.xlabel("declination [°]", fontsize=15)
#    plt.ylabel("prob", fontsize=15)
#    plt.plot(df["bins (dec)"][element], df['prob (dec)'][element])
#    plt.hist(df["bins (dec)"][element], color="green", weights=df['prob (dec)'][element], bins=5, stacked=True)

In [ ]:
def Fit( minimum=31, maximum=39, bins=20, weighted=False):
    selected=df.loc[(df['mean declination'] > minimum) & (df['mean declination'] <maximum)]
    if weighted==True:
        data=list(selected["mean declination"])
    else:
        l=list(selected["declination"])
        data = [item for sublist in l for item in sublist]
    if len(data)>0:
        plt.figure(figsize=(12,10))
        # best fit of data
        (mu, sigma) = norm.fit(data)
        # the histogram of the data
        n, bins, patches = plt.hist(data,  bins=bins, normed=1, facecolor='green', alpha=0.75)
        # add a 'best fit' line
        y = mlab.normpdf(bins, mu, sigma)
        #l = plt.plot(bins, y, 'r--', linewidth=2)
        #plot
        plt.xlabel("declination [°]")
        plt.ylabel("prob")
        #plt.title('declination=%.3f°, $\sigma=%.3f$' %(mu,sigma))
        plt.show()
#Fit(minimum=20, maximum=40, bins=6, weighted=True)

In [9]:
c1=Citable("values_n200_th10_sig0.2_start0.5.json", formats="local", project="kreisgrabenanlage")
df=c1.digitalresource()
c2=Citable("~/ResearchCloud/Projects/kreisgrabenanlage/data/test.json", formats="local", project="kreisgrabenanlage")

In [11]:
#c2.write(df)